# Advent of Code 2020

login is **Github** credentials

https://adventofcode.com/2020/

In [1]:
print("hello santa")

hello santa


## Imports Setup Etc

In [2]:
import numpy as np
import pandas as pd
import re

## Day 1    

**Part 1**: Find the 2 numbers that add to 2020 report the product of those numbers

**Part 2**: Same as part 1 but with 3 numbers

In [3]:
def day1():

    ## load in data
    with open("input01.txt") as f: # Read file ('with' closes when done)
        info = f.read()
    info = info.split()            # Get list of values (as strings)
    info = [int(s) for s in info]  # Convert strings to ints

    ## Sort data
    info.sort()  #sort the data in place. 

    ### part 1

    ## Check sums on the sorted info
        ## start with smallest number add largest number until you get smaller than 2020 
        ## at that point move to second largest number
    n1 = 0
    n2 = 0
    jj = len(info)-1

    for ii in range(len(info)): 
        while (info[ii] + info[jj]) >= 2020 and n1 == 0:
            if info[ii] + info[jj] == 2020:
                n1 = info[ii]  # grab number we are looking for
                n2 = info[jj]  # grab number
                jj = 1         # break while loop
            jj -= 1
        jj += 1
    
    print("Part 1:")
    print("{} + {} = {}".format(n1,n2,n1+n2))
    print("{} x {} = {}".format(n1,n2,n1*n2))

    ### part 2

    ## similar to part 1, but we'll start one at the lowest, another right above, a third at the end. 
        ## move one at end down until gt 2020 then move second one up and try again (similar to above)
        ## if it's too big at jj==1, then increment ii and start over
    n1 = 0
    n2 = 0
    n3 = 0
    ii = 0
    jj = 1
    kk = len(info)-1
    while n1 == 0:
        while jj < kk and n1 == 0:
            while (info[ii] + info[jj] + info[kk]) >= 2020 and n1 == 0:
    #             print("{} + {} + {} = {}".format(info[ii],info[jj],info[kk],info[ii]+info[jj]+info[kk]))
                if info[ii] + info[jj] + info[kk] == 2020:
                    n1 = info[ii]  # grab number we are looking for
                    n2 = info[jj]  # grab number
                    n3 = info[kk]  # grab number
                kk -= 1
            jj += 1
            kk += 1
        ii += 1
        jj = ii + 1
        kk = len(info)-1

    print("\nPart 2")
    print("{} + {} + {} = {}".format(n1,n2,n3,n1+n2+n3))
    print("{} x {} x {} = {}".format(n1,n2,n3,n1*n2*n3))

day1()

Part 1:
62 + 1958 = 2020
62 x 1958 = 121396

Part 2
117 + 426 + 1477 = 2020
117 x 426 x 1477 = 73616634


## Day 2

**Part 1**: Count the number of valid passwords. Valid passwords are ones where the number of occurences of the letter indicated are between the min and max number of occurences shown.

**Part 2**: New definition of valid, the letter indicated needs to occur at exactly one of the 2 indicies indicated. Note that this is a "one based" indexing system for the problem.

#### Resources

* StackOverflow XOR in python
  * https://stackoverflow.com/questions/432842/how-do-you-get-the-logical-xor-of-two-variables-in-python
* Documentation for python re library and usage
  * https://docs.python.org/3/library/re.html
* Suuuper Helpful video from Corey Schafer on regular expressions
  * The part that I'm most interested in in about 45 minutes in
  * https://www.youtube.com/watch?v=K8L6KVGG-7o


In [4]:
def day2():
    
    ### Part 1
    
    ## Load in data
    with open("input02.txt") as f: # Read file ('with' closes when done)
        raw = f.read()
    
    ## Split up rows into (min, max, letter, password)
    pat = re.compile("(\d+)-(\d+)\s([a-z]):\s([a-z]+)")
    reres = pat.findall(raw)

    ## Make a list to keep track of valid passwords
    isValid = [False]*len(reres)
    
    ## Check validity
    rcnt = 0
    for row in reres:
        ccnt = 0
        for c in row[3]:
            if c == row[2]:
                ccnt += 1
        isValid[rcnt] = ccnt >= int(row[0]) and ccnt <= int(row[1])
        rcnt += 1
    
    print("Part 1")
    print("There are {} valid passwords".format(sum(isValid)))
    
    ### Part 2
        ## Reusing a lot of part 1. Only validity check changes.
    
    ## Make a list to keep track of valid passwords
    isValid = [False]*len(reres)
    
    ## Check validity
    rcnt = 0
    for row in reres:
        ## see if characters at required indices are valid
            ## the "-1" converts the 1 based idx given to our zero based index
            ## != functions as xor operator comparing booleans
        isValid[rcnt] = ((row[3][int(row[0])-1] == row[2]) != 
                         (row[3][int(row[1])-1] == row[2]))
        rcnt += 1
    
    print("\nPart 2")
    print("There are {} valid passwords".format(sum(isValid)))
            
day2()

Part 1
There are 536 valid passwords

Part 2
There are 558 valid passwords


## Day 3

**Part 1**: It's a long description. Essentially count the number of `#` you hit onthe way down moving over 3 cells and down 1 cell. Assume that the pattern given is repeated indefinite number of times horizontally.

This basically looks like it's going to be pretty easy with good use of the modulo function.

**Part 2**: Try again with some different slopes. Report the product of those slopes. Specifically, try:

* Right 1, down 1.
* Right 3, down 1. (This is the slope you already checked.)
* Right 5, down 1.
* Right 7, down 1.
* Right 1, down 2


In [5]:
def day3_help(over = 3, down = 1):
    
    ## It's a little clunky to hardcode the file read but... whatever...
    ## Load in data
    with open("input03.txt") as f: # Read file ('with' closes when done)
        raw = f.read()
    
    ## Split data into a list of rows
    res = raw.split()
    
    ## Work with smaller set initially easier to QC
    # res = res[:10]
    
    ## Set up some control and counter variables
    nrow = len(res)     # variable for number of rows
    ncol = len(res[0])  # variable for num columns (all rows have same)
    colCnt = 0          # track number of collisions
    
    for ii in range(nrow):
        if ii%down == 0 and res[ii][(ii//down)*over%ncol] == "#":
            colCnt += 1
    
    return colCnt

def day3():
    
    ### Part 1
    
    over = 3
    down = 1
    
    colCnt = day3_help(over,down)

    print("Part 1")
    print("You will hit {} trees at an angle of {} to {}".format(colCnt,over,down))
    
    ### Part 2
    
    slopes = [[1,1],
              [3,1],
              [5,1],
              [7,1],
              [1,2]]
    
    cols = [0]*5  # Create empty list to store nubmer of collisions each
    product = 1
    cnt = 0
    for slope in slopes:
        cols[cnt] = day3_help(slope[0],slope[1])
        product *= cols[cnt]
        cnt += 1
    
    print("\nPart 2")
    print(cols)
    print(np.prod(cols))
    print("The product of the collisions is {}".format(product))
            
day3()

Part 1
You will hit 218 trees at an angle of 3 to 1

Part 2
[77, 218, 65, 82, 43]
3847183340
The product of the collisions is 3847183340


## Day 4

**Part 1**: Checking if passport info has all required fields

**Part 2**: More validation required

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

* byr (Birth Year) - four digits; at least 1920 and at most 2002.
* iyr (Issue Year) - four digits; at least 2010 and at most 2020.
* eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
* hgt (Height) - a number followed by either cm or in:
* If cm, the number must be at least 150 and at most 193.
* If in, the number must be at least 59 and at most 76.
* hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
* ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
* pid (Passport ID) - a nine-digit number, including leading zeroes.
* cid (Country ID) - ignored, missing or not.

In [6]:
def day4():
    
    with open("input04.txt") as f:
        raw = f.read()
    
    ### Part 1
    
    ## Setup 
    
    ## Required Fields
    req = ["byr","iyr","eyr","hgt","hcl","ecl","pid"] # 'cid' not required
    
    ## Split up and check for Required Feilds
    
    ## Find 3 ltr strings followed by ":" and anum chrs 
        ## Keep initial ltrs as group 1. Second string as grp 2.
    pat = re.compile("([a-z]{3}):(\S+)") 
    
    ## Actually split and run.
    res = raw.split(sep = "\n\n")             # First split by passport
    valid = [True] * len(res)                 # Make collector for validity
    for ii in range(len(res)):
        res[ii] = dict(pat.findall(res[ii]))  # For each passport break up the info and put into dict
        for jj in range(len(req)):
            if not(req[jj] in res[ii]):
                valid[ii] = False
    
    print("Part 1")
    print("There are {} valid passports".format(sum(valid)))
    
    ### Part 2 
    
    valid = [True]*len(res)
    
    for ii in range(len(res)):
        
        ## Validate birth year
        if "byr" in res[ii]: 
            if int(res[ii]["byr"]) < 1920 or int(res[ii]["byr"]) > 2002:
                valid[ii] = False
        else:
            valid[ii] = False
        
        ## Validate issue year
        if "iyr" in res[ii]:
            if int(res[ii]["iyr"]) < 2010 or int(res[ii]["iyr"]) > 2020:
                valid[ii] = False
        else: 
            valid[ii] = False
        
        ## Validate expiry year
        if "eyr" in res[ii]:
            if int(res[ii]["eyr"]) < 2020 or int(res[ii]["eyr"]) > 2030:
                valid[ii] = False
        else: 
            valid[ii] = False
        
        ## Validate height
        if "hgt" in res[ii]:
            if re.search("([0-9]+)(in|cm)",res[ii]["hgt"]) != None:
                (meas, unit) = re.findall("([0-9]+)(in|cm)",res[ii]["hgt"])[0]
                if unit == "cm" and (int(meas) < 150 or int(meas) > 193):
                    valid[ii] = False
                if unit == "in" and (int(meas) < 59 or int(meas) > 76):
                    valid[ii] = False
            else:
                valid[ii] = False
        else: 
            valid[ii] = False
        
        ## Validate haircolor
        if "hcl" in res[ii]:
            if re.search("#[0-9a-f]",res[ii]["hcl"]) == None:
                valid[ii] = False
        else:
            valid[ii] = False
        
        ## Validate eyecolor
        if not(res[ii].get("ecl") in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]):
            valid[ii] = False
            
        ## Validate passport id
        if "pid" in res[ii]:
            if re.search("\A\d{9}$",res[ii]["pid"]) == None:
                valid[ii] = False
        else:
            valid[ii] = False
    
    print("\nPart 2")
    print("With more scrutiny, there are {} valid passports".format(sum(valid)))

day4()

Part 1
There are 254 valid passports

Part 2
With more scrutiny, there are 184 valid passports


## Day 5

**Part 1**: Take binary "seat row and column" convert them to integer. Use thes integers to find unique id

**Part 2**: Find the missing id (ids don't start at 0)

**Refernces*:

* StackOverflow Post on converting binary strings to integers
    * https://stackoverflow.com/questions/8928240/convert-base-2-binary-number-string-to-int/8928256#8928256

In [7]:
def day5():
    
    with open("input05.txt") as f:
        raw = f.read()
    
    pat = re.compile("([F|B]+)([R|L]+)")
    res1 = pat.findall(raw)
    
    ### Part 1
    
    res2 = [0] * len(res1)        # Build list of zeros
    for ii in range(len(res1)):
        rnum = int(res1[ii][0].replace("F","0").replace("B","1"),2) # pull our row num
        cnum = int(res1[ii][1].replace("L","0").replace("R","1"),2) # pull out col num
        res2[ii] = rnum * 8 + cnum                                  # grab id number
    
    print("Part 1")
    print("The largest id number is {}".format(max(res2)))
    
    ### Part 2
    
    res2.sort()  # sort the ids in place
    
    res3 = 0
    cnt = 0
    while res3 == 0:
        if res2[cnt] != res2[cnt+1]-1:
            res3 = res2[cnt]+1
        cnt += 1
    
    print("\nPart 2")
    print("Your seat is {}".format(res3))
            
day5()

Part 1
The largest id number is 848

Part 2
Your seat is 682


## Day 6

**Part 1**: count number of questions that anyone ina group answered yes to

I'm going to dump them all in set objects. I just need to strip out newline characters

**Part 2**: count the number of questions that everyone in a group answered yes to.


In [8]:
def day6():
    
    with open("input06.txt") as f:
        raw = f.read()
    
    ### Part 1
    
    res1 = raw.split("\n\n")
    
    res2 = [0]*len(res1)
    
    for ii in range(len(res1)):
        res1[ii] = res1[ii].replace("\n","")
        res2[ii] = len(set(res1[ii]))

    print("Part 1")
    print("The sum of the number of positive answers (per group is) {}".format(sum(res2)))
    
    ### Part 2
    
    res3 = [r.split("\n") for r in raw.split("\n\n")]
    
    res4 = [0]*len(res3)
    
    
    ## dictionary based soultion Unfinished. I like a set based solution better. 
#     for ii in range(len(res3)):
        
#         resdic = dict()                 # dict to hold intermediate results
#         for jj in range(len(res3[ii])):
#             for kk in range(len(res3[ii][jj])):
#                 c = len(res3[ii][jj][kk])
#                     count = resdic.get(c)
#                     resdic[c] = 1 if c==None else c + 1

    for ii in range(len(res3)):
        rptset = set(res3[ii][0])
        for jj in range(1,len(res3[ii])):
            rptset = rptset.intersection(set(res3[ii][jj]))
        res4[ii] = len(rptset)
    
    print("\nPart 2")
    print("The sum of the number of positive answers (per group is) {}".format(sum(res4)))
            
day6()

Part 1
The sum of the number of positive answers (per group is) 6778

Part 2
The sum of the number of positive answers (per group is) 3406


## Day 7

**Part 1**: There is a heirarchy of what types of bags can fit into other bags. Given that we have a shiny gold bag, we want to find out how many different types of bags could contain it. This is apparently a directed acyclic graph DAG. I looked into ways of representing graphs in python. The dictionary approach seems very elegant. 

**Part 2**: Solve a more nuanced version of the same puzzle

**References**:

* Building and Representing a graph in Python.
  * https://www.python.org/doc/essays/graphs/

In [11]:
def day7():
    
    with open("input07lite.txt") as f:
#     with open("input07.txt") as f:
        raw = f.read()
    
    print(raw)
    
    pat = re.compile("(\w+ \w+) bags contain \d\d? (\w+ \w+) bags?, \d\d? (\w+ \+)")
    
    res1 = pat.findall(raw)
    
    print(res1)
    
    ### Part 1

    print("Part 1")
    print("You will hit {} trees at an angle of {} to {}".format(420,6,9))
    
    ### Part 2
    
    print("\nPart 2")
    print("The product of the collisions is {}".format("cool explosions"))
            
day7()

light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
[('light red', 'bright white', 'muted yellow'), ('dark orange', 'bright white', 'muted yellow'), ('muted yellow', 'shiny gold', 'faded blue'), ('shiny gold', 'dark olive', 'vibrant plum'), ('dark olive', 'faded blue', 'dotted black'), ('vibrant plum', 'faded blue', 'dotted black')]
Part 1
You will hit 420 trees at an angle of 6 to 9

Part 2
The product of the collisions is cool explosions


## Scratch

In [ ]:
a = set("aaabc")
print(a)
b = set("aabca\ncba  cac")
print(b)
for ii in range(1,4):
    print(ii)

## Day n

**Part 1**: Solve a puzzle

**Part 2**: Solve a more nuanced version of the same puzzle


In [ ]:
def dayN():
    
    with open("inputN.txt") as f:
        raw = f.read()
    
    print(raw)
    
    ### Part 1

    print("Part 1")
    print("You will hit {} trees at an angle of {} to {}".format(420,6,9))
    
    ### Part 2
    
    print("\nPart 2")
    print("The product of the collisions is {}".format("cool explosions"))
            
dayN()

## Backup

Create alternate backup versions that are easier to diff in git and easier to view in a pinch

In [ ]:
#Create a backup of the jupyter notebook in a format for where changes are easier to see.
!jupyter nbconvert solutions.ipynb --to="python" --output="backups/solutions"
!jupyter nbconvert solutions.ipynb --to markdown --output="backups/solutions"

# Also archiving this bad boy
!jupyter nbconvert solutions.ipynb --to html --output="backups/solutions"